In [1]:
import pandas as pd
import time
import re
import pathlib
import os
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [3]:
#https://chromedriver.chromium.org/downloads

In [4]:
driver_path = os.path.join(pathlib.Path().absolute(), 'chrome_driver\\chromedriver.exe')
driver = webdriver.Chrome(executable_path = driver_path, options=options)
driver.get('https://pv.servelelecciones.cl/')

In [5]:
%%time
#PESTAÑA SUPERIOR: "Participación"
step1 = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[3]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step1))).click()

#PANEL LATERAL IZQUIERDO: "En Chile"
step2 = '/html/body/div[2]/div[1]/div[2]/div/ul/li[3]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step2))).click()

print('Introducir el número del distrito que desea revisar (1-28)')
distrito = int(input())
if distrito in range(1,29):
    #PANEL LATERAL IZQUIERDO: "Distrito n°"
    step3 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[6]/option[{distrito + 1}]'
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step3))).click()
    
    time.sleep(5)
    
    df = pd.DataFrame([])
    
    #PANEL LATERAL IZQUIERDO: "Comunas"
    df1 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
    df1 = df1[df1 != '\n'].dropna(axis=0).reset_index(drop=True)
    df1[0] = df1[0].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
    
    #PANEL LATERAL IZQUIERDO: "Circunscripciones Electorales"
    for i in range(len(df1)):
        step4 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[9]/option[{i+2}]'
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step4))).click()
        time.sleep(3)
        df2 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
        df2 = df2[df2 != '\n'].dropna(axis=0).reset_index(drop=True)
        df2[0] = df2[0].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
        
        #PANEL LATERAL IZQUIERDO: "Locales de Votación"
        for j in range(len(df2)):
            step5 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[10]/option[{j+2}]'
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step5))).click()
            time.sleep(3)
            df3 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
            df3 = df3[df3 != '\n'].dropna(axis=0).reset_index(drop=True)
            df3[0] = df3[0].apply(lambda x: "".join(re.split("( )", x)[0:len(re.split("( )", x))-8]))
            
            #MESAS DE VOTACIÓN
            for k in range(len(df3)):
                step6 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[11]/option[{k+2}]'
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step6))).click()
                time.sleep(3)
                df4 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
                df4 = df4[df4 != '\n'].dropna(axis=0).reset_index(drop=True)
                df4 = df4[0].str.split(expand=True)
                df4[5] = df3[0][k]
                df4[6] = df1[0][i]
                df4[7] = df2[0][j]
                df = pd.concat([df,df4])
    for m in range(1,4):
        df[m] = df[m].astype(int)
    df[4] = df[4].apply(lambda x: x.replace(',', '.')).astype('float64')
    columns = ['MESA', 'TOTALMESAS' , 'ELECTORES', 'EMITIDOS', 'PORCENTAJE', 'COLEGIO', 'COMUNA', 'CIRCUNSCRIPCION']
    df.columns = columns
    df = df.dropna(axis=0).reset_index(drop=True)
    df.to_csv(f'Dist{distrito}_Pleb2020.csv', index=False)
    path1 = os.path.join(pathlib.Path().absolute(), f'Distrito{distrito}')
    os.mkdir(path1)
    df.to_csv(f'Distrito{distrito}\Dist{distrito}_Pleb2020.csv', index=False)
    
else:
    sys.exit(f'No existe el distrito n°: {distrito}, elija uno entre 1-28')

Introducir el número del distrito que desea revisar (1-28)
2
Wall time: 4min 39s


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MESA             865 non-null    object 
 1   TOTALMESAS       865 non-null    int32  
 2   ELECTORES        865 non-null    int32  
 3   EMITIDOS         865 non-null    int32  
 4   PORCENTAJE       865 non-null    float64
 5   COLEGIO          865 non-null    object 
 6   COMUNA           865 non-null    object 
 7   CIRCUNSCRIPCION  865 non-null    object 
dtypes: float64(1), int32(3), object(4)
memory usage: 44.1+ KB


In [7]:
df

,MESA,TOTALMESAS,ELECTORES,EMITIDOS,PORCENTAJE,COLEGIO,COMUNA,CIRCUNSCRIPCION
0,18M,1,344,174,50.58,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,ALTO HOSPICIO,ALTO HOSPICIO
1,19M,1,343,189,55.10,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,ALTO HOSPICIO,ALTO HOSPICIO
2,20M,1,337,178,52.82,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,ALTO HOSPICIO,ALTO HOSPICIO
3,21M,1,343,181,52.77,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,ALTO HOSPICIO,ALTO HOSPICIO
4,22M,1,339,183,53.98,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,ALTO HOSPICIO,ALTO HOSPICIO
...,...,...,...,...,...,...,...,...
860,3V,1,296,111,37.50,LICEO ALCALDE SERGIO GONZALEZ GUTIERREZ,POZO ALMONTE,POZO ALMONTE
861,4V,1,294,144,48.98,LICEO ALCALDE SERGIO GONZALEZ GUTIERREZ,POZO ALMONTE,POZO ALMONTE
862,5V,1,287,130,45.30,LICEO ALCALDE SERGIO GONZALEZ GUTIERREZ,POZO ALMONTE,POZO ALMONTE
863,6V-8V,1,442,164,37.10,LICEO ALCALDE SERGIO GONZALEZ GUTIERREZ,POZO ALMONTE,POZO ALMONTE
